<a href="https://colab.research.google.com/github/SmartEngineer1/mlbootcamp2025flc/blob/main/9_MLBootcamp_FinalProjectMohsinMohammad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Bootcamp 2025

### Final Project: Train a Deep Learning model to identify Grocery item

In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from google.colab import drive #for the mounting
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split #for actual splitting to be done
import numpy as np
from PIL import Image
import os
import csv
from datetime import datetime

In [43]:
drive.mount('/content/drive/', force_remount=True)
!ls /content/drive/MyDrive/Datasets/
!ls /content/drive/MyDrive/Datasets/GroceryStoreDataset/

Mounted at /content/drive/
GroceryStoreDataset
GroceryStoreDataset


In [44]:
# NOTE: Create directory 'Datasets/GroceryStoreDataset', unzip the shared dataset in it and mount the Google Drive
# The original dataset used is: https://www.kaggle.com/datasets/validmodel/grocery-store-dataset?resource=download and it has been reduced further for our use-case
data_dir = '/content/drive/MyDrive/Datasets/GroceryStoreDataset'  # Mount dataset in Google Drive

In [45]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


In [46]:
def extract_class_names(csv_file_path):
    """
    Extracts class names from a CSV file and returns them as a list.


    Args:
        csv_file_path (str): The path to the CSV file.

    Returns:
        tuple: A tuple containing a list of class names and the number of classes.
               Returns (None, 0) if the file does not exist or an error occurs.
    """
    try:
        with open(csv_file_path, 'r') as file:
            reader = csv.reader(file)
            next(reader)  # Skip header row if it exists
            class_names = [row[2] for row in reader]  # Assuming class names are in the first column
            class_names = sorted(set(class_names))
        return class_names, len(class_names)

    except FileNotFoundError:
        print(f"Error: File '{csv_file_path}' not found.")
        return None, 0
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, 0

In [7]:
#


In [47]:
# Data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

def fix_channel(matrix):
  return matrix.repeat(3, 1, 1) # 1s are due to height and weight

transform = transforms.Compose([
    #transforms.Grayscale(channel = 3), #cause im getting the error message
    transforms.ToTensor(),
    transforms.Lambda(fix_channel),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

print(f"Train dataset classes: {train_loader.dataset.classes}")
print(f"Test dataset classes: {test_loader.dataset.classes}")

splitter_web = {
    #something gets in here
}

taser_test = {}
grenade_train = {}


total_class_train = {}
total_class_test = {}

for image, label in train_dataset:
  if label in total_class_train:
      total_class_train[label] += 1

  else:
      total_class_train[label] = 1

  if label not in splitter_web:
      splitter_web[label] = []
  splitter_web[label].append(image)


print(f"Trained class {total_class_train}")

for image, label in test_dataset:
  if label in total_class_test:
      total_class_test[label] += 1

  else:
      total_class_test[label] = 1

  if label not in splitter_web:
      splitter_web[label] = []
  splitter_web[label].append(image)

#THHHHHE SPLLLLIIIIITTT
for label, imagelist in splitter_web.items():
  grenade_trainer, taser_tester = train_test_split(imagelist, test_size = 0.5, random_state = 42)
  taser_test[label] = taser_tester
  grenade_train[label] = grenade_trainer

print(f"Tested class {total_class_test}")

#If i called the confusion matrix and the rest here it will work



Train dataset classes: ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
Test dataset classes: ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
Trained class {5: 5421, 0: 5923, 4: 5842, 1: 6742, 9: 5949, 2: 5958, 3: 6131, 6: 5918, 7: 6265, 8: 5851}
Tested class {7: 1028, 2: 1032, 1: 1135, 0: 980, 4: 982, 9: 1009, 5: 892, 6: 958, 3: 1010, 8: 974}


In [48]:
# Identify that all folders are accessible from mounted Google Drive
data_dir = '/content/drive/MyDrive/Datasets/GroceryStoreDataset'
for x in ['train', 'test']:
  path_new = os.path.join(data_dir, x)
  print(path_new)

  if (os.path.exists(path_new)):
    for folder in os.listdir(path_new):
      folder_path = os.path.join(path_new, folder)
      print(folder_path)
  else:
    print(f"doesn't work I'm afraid")


/content/drive/MyDrive/Datasets/GroceryStoreDataset/train
doesn't work I'm afraid
/content/drive/MyDrive/Datasets/GroceryStoreDataset/test
doesn't work I'm afraid


In [49]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'test']}  # Assuming you have train and test folders
dataloaders = {x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=2)
              for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

print(f"Dataset sizes: {dataset_sizes}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Datasets/GroceryStoreDataset/train'

In [ ]:
class_names = image_datasets['train'].classes

print(class_names)
print(len(class_names))

class_names = image_datasets['test'].classes

print(class_names)
print(len(class_names))

num_classes = len(class_names)

In [50]:
# Load pre-trained EfficientNetB4
model = models.efficientnet_b4(pretrained=True)

# Modify the classifier
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B4_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B4_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-23ab8bcd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-23ab8bcd.pth
100%|██████████| 74.5M/74.5M [00:01<00:00, 72.5MB/s]


NameError: name 'num_classes' is not defined

In [ ]:
def predict_image(image_path, model, class_names):
  img = Image.open(image_path).convert('RGB')
  img_t = data_transforms['test'](img).unsqueeze(0)
  img_t = img_t.to(device)
  model.eval()
  with torch.no_grad():
    out = model(img_t)
    _, index = torch.max(out, 1)
    percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
    print(f"Predicted Class: {class_names[index[0]]}, Confidence: {percentage[index[0]].item():.2f}%")

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=25):
    history = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': []}
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        now = datetime.now()
        print(now.strftime("%Y-%m-%d %H:%M:%S"))
        print('-' * 10)

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0

            i = 0
            for inputs, labels in dataloaders[phase]:
                i += 1
                if i % 10 == 0:
                    print(f"Batch {i} of {len(dataloaders[phase])}")
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc)

    return model, history


def result():
    model.eval()
    correct = 0
    total = 0
    model.to(device)
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in test_loader:
          data, target = data.to(device), target.to(device)
          output = model(data)

          _, predicted = torch.max(output.data, 1)
          total += target.size(0)
          correct += (predicted == target).sum().item()
          all_preds.extend(predicted.cpu().numpy())
          all_targets.extend(target.cpu().numpy())


    #to make it more easier to read for myself
    myacc = correct / total
    myfone = f1_score(all_targets, all_preds, average=None)
    myprec = precision_score(all_targets, all_preds, average=None)
    myrecall = recall_score(all_targets, all_preds, average=None)
    mymat = confusion_matrix(all_targets, all_preds)
    return myacc, myfone, myprec, myrecall, mymat



donemodel, donehistory = train_model(model, criterion, optimizer, num_epochs=25)
print(f"The model: {donemodel}")
print(f"The history: {donehistory}")



doneacc, donefone, doneprec, donerecall, donemat = result()
print(f"The Accuracy: {doneacc}")
print(f"The F1 score: {donefone}")
print(f"The Precision score: {doneprec}")
print(f"The Recall score: {donerecall}")
print(f"The Confusion matrix: {donemat}")

##time for the 6x4 thing. lots or resources spent reviewring and studying lol
model.eval()
model.to(device)
with torch.no_grad():
    for data, label in test_loader:
      output = model(data)
      _, predicted = torch.max(output.data, 1)
      grid, axes = plt.subplots(6, 4, figsize = (12, 18))

      axes = axes.flatten()

      for i in range(24):
        axes[i].imshow(data[i].cpu().permute(1, 2, 0).numpy())
        axes[i].set_title(f"Real {label[i].item()} vs predicted {predicted[i].item()}")



In [1]:
# Before training
predict_image('/content/drive/MyDrive/Datasets/GroceryStoreDataset/test/Mango/Mango_002.jpg', model, class_names)
predict_image('/content/drive/MyDrive/Datasets/GroceryStoreDataset/test/Pineapple/Pineapple_021.jpg', model, class_names)


NameError: name 'predict_image' is not defined

In [20]:
model, history = train_model(model, criterion, optimizer, num_epochs=18)


NameError: name 'criterion' is not defined

In [21]:
metrics = {
    'train_loss': history['train_loss'],
    'train_acc': [t.cpu().item() for t in history['train_acc']],
    'test_loss': history['test_loss'],
    'test_acc': [t.cpu().item() for t in history['test_acc']]
}

print(metrics)

# Plot the training history
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(metrics['train_loss'], label='Train Loss')
plt.plot(metrics['test_loss'], label='Test Loss')
plt.legend()
plt.title('Loss')

plt.subplot(1, 2, 2)
plt.plot(metrics['train_acc'], label='Train Accuracy')
plt.plot(metrics['test_acc'], label='Test Accuracy')
plt.legend()
plt.title('Accuracy')
plt.show()

NameError: name 'history' is not defined

In [ ]:
# save the trained model
torch.save(model, '/content/drive/MyDrive/Datasets/GroceryStoreDataset/complete_model.pth')


In [ ]:
# After training
predict_image('/content/drive/MyDrive/Datasets/GroceryStoreDataset/test/Mango/Mango_002.jpg', model, class_names)
predict_image('/content/drive/MyDrive/Datasets/GroceryStoreDataset/test/Pineapple/Pineapple_021.jpg', model, class_names)


### Project:

1. Perform exploratory data analysis on the 'train' and 'test' datasets to calculate class imbalance (by comparing 'samples per class' across all the classes)
2. Print confusion matrix, precision, recall and f1-score
3. Show a grid of 6x4 images, with actual and predicted class for each of those


### Bonus Project:

1. Allow user to input the items they shopped using images, use model to identify grocery item based on confidence threshold. If confidence is low, ask user to manually input the item.
2. Update the digital grocery cart
3. Process the transaction by generating a transaction receipt
